In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from os import path

In [3]:
security_projects=['AES_Flash', 'TrustZone_Flash', 'No_Security_Flash']

In [4]:
aes_encrypt_charge = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_encrypt_charge.p"), "rb"))
aes_decrypt_charge = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_decrypt_charge.p"), "rb"))
aes_flash_write_charge = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_write_charge.p"), "rb"))
aes_flash_read_charge = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_read_charge.p"), "rb"))
aes_encrypt_time = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_encrypt_time.p"), "rb"))
aes_decrypt_time = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_decrypt_time.p"), "rb"))
aes_flash_write_time = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_write_time.p"), "rb"))
aes_flash_read_time = pickle.load(open(path.join("Security", "AES_Flash", "aes_flash_read_time.p"), "rb"))
aes = [aes_encrypt_charge, aes_decrypt_charge, aes_flash_write_charge, aes_flash_read_charge, aes_encrypt_time, aes_decrypt_time, aes_flash_write_time, aes_flash_read_time]
for i in aes:
    print(len(i), len(i)*16)

FileNotFoundError: [Errno 2] No such file or directory: 'Security\\AES_Flash\\aes_flash_encrypt_charge.p'

In [5]:
flash_write_charge = pickle.load(open(path.join("Security", "No_Security_Flash", "no_security_flash_write_charge.p"), "rb"))
flash_read_charge = pickle.load(open(path.join("Security", "No_Security_Flash", "no_security_flash_read_charge.p"), "rb"))
flash_write_time = pickle.load(open(path.join("Security", "No_Security_Flash", "no_security_flash_write_time.p"), "rb"))
flash_read_time = pickle.load(open(path.join("Security", "No_Security_Flash", "no_security_flash_read_time.p"), "rb"))
no_security = [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]
for i in no_security:
    print(len(i))

5999
5999
5999
5999


In [6]:
nsc_store_charge = pickle.load(open(path.join("Security", "TrustZone_Flash", "trustzone_flash_store_charge.p"), "rb"))
nsc_load_charge = pickle.load(open(path.join("Security", "TrustZone_Flash", "trustzone_flash_load_charge.p"), "rb"))
nsc_store_time = pickle.load(open(path.join("Security", "TrustZone_Flash", "trustzone_flash_store_time.p"), "rb"))
nsc_load_time = pickle.load(open(path.join("Security", "TrustZone_Flash", "trustzone_flash_load_time.p"), "rb"))
trustzone = [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
for i in trustzone:
    print(len(i))

6000
6000
6000
6000


In [7]:
min_num = 6000-1

In [8]:
j_scale = 1e6
voltage = 3.33

In [9]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
aes_chkp = []
aes_rest = []
aes_x = []
for i in range(0, min_num//STEP_SIZE):
    aes_x.append(MIN_NUM_BYTES + i*STEP_SIZE)
    aes_chkp.append((aes_encrypt_charge[i] + aes_flash_write_charge[i]) * voltage * j_scale)
    aes_rest.append((aes_decrypt_charge[i] + aes_flash_read_charge[i]) * voltage * j_scale)

NameError: name 'aes_encrypt_charge' is not defined

In [34]:
no_security_chkp = []
no_security_rest = []
no_security_x = []
for i in range(0, min_num):
    no_security_x.append(i+1)
    no_security_chkp.append(flash_write_charge[i] * voltage * j_scale)
    no_security_rest.append(flash_read_charge[i] * voltage * j_scale)

In [35]:
trustzone_chkp = []
trustzone_rest = []
trustzone_x = []
for i in range(0, min_num):
    trustzone_x.append(i+1)
    trustzone_chkp.append(nsc_store_charge[i] * voltage * j_scale)
    trustzone_rest.append(nsc_load_charge[i] * voltage * j_scale)

In [36]:
every = 500
everies= [every//MBEDTLS_AES_BLOCK_SIZE*2, every//MBEDTLS_AES_BLOCK_SIZE, every, every*2, every*2, every]
colors = ['', '', '', '', '', ''] #'k'
linestyles = [(0, (1.0, 0)), (0, (1.0, 0.5)), (0, (2.0, 1.0)), (0, (3.0, 1.5)), (0, (4.0, 2.0)), (0, (5.0, 1.5))]
markers = ['v', '^', 'o', 'D', '<', '>']

In [37]:
fig_energy = plt.figure(figsize=(6, 4))
fig_energy.canvas.set_window_title("Analysis Energy")

In [38]:
fig_energy.clf()
ax1 = fig_energy.add_subplot(1, 1, 1)
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [$\mu$J]') # $\mu$

Text(0, 0.5, 'Energy [$\\mu$J]')

In [39]:
lines = []
lines += ax1.plot(aes_x, aes_chkp, label='AES Checkpoint')
lines += ax1.plot(aes_x, aes_rest, label='AES Restore')
lines += ax1.plot(no_security_x, no_security_chkp, label='No Security Checkpoint')
lines += ax1.plot(no_security_x, no_security_rest, label='No Security Restore')
lines += ax1.plot(trustzone_x, trustzone_chkp, label='TrustZone Checkpoint')
lines += ax1.plot(trustzone_x, trustzone_rest, label='TrustZone Restore')
for i in range(len(lines)):
    lines[i].set_markevery(everies[i])
    lines[i].set_linestyle(linestyles[i])
    lines[i].set_marker(markers[i])
#     lines[i].set_color(colors[i])

ax1.legend(handles=lines)
# ax1.set_yscale('log')
ax1.grid()
fig_energy.tight_layout()
fig_energy.show()

In [19]:
t_scale = 1e3
# min_num = 512

In [20]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
aes_time_chkp = []
aes_time_rest = []
aes_time_x = []
for i in range(0, min_num//STEP_SIZE):
    aes_time_x.append(MIN_NUM_BYTES + i*STEP_SIZE)
    aes_time_chkp.append((aes_encrypt_time[i] + aes_flash_write_time[i]) * t_scale)
    aes_time_rest.append((aes_decrypt_time[i] + aes_flash_read_time[i]) * t_scale)

In [21]:
no_security_time_chkp = []
no_security_time_rest = []
no_security_time_x = []
for i in range(0, min_num):
    no_security_time_x.append(i+1)
    no_security_time_chkp.append(flash_write_time[i] * t_scale)
    no_security_time_rest.append(flash_read_time[i] * t_scale)

In [22]:
trustzone_time_chkp = []
trustzone_time_rest = []
trustzone_time_x = []
for i in range(0, min_num):
    trustzone_time_x.append(i+1)
    trustzone_time_chkp.append(nsc_store_time[i] * t_scale)
    trustzone_time_rest.append(nsc_load_time[i] * t_scale)

In [29]:
fig_time = plt.figure(figsize=(6, 4))
fig_time.canvas.set_window_title("Analysis Time")

In [30]:
fig_time.clf()
ax2 = fig_time.add_subplot(1, 1, 1)
ax2.set_xlabel('Checkpoint Size [Bytes]')
ax2.set_ylabel('Time [ms]') # $\mu$

Text(0, 0.5, 'Time [ms]')

In [31]:
lines = []
lines += ax2.plot(aes_time_x, aes_time_chkp, label='AES Checkpoint')
lines += ax2.plot(aes_time_x, aes_time_rest, label='AES Restore')
lines += ax2.plot(no_security_time_x, no_security_time_chkp, label='No Security Checkpoint')
lines += ax2.plot(no_security_time_x, no_security_time_rest, label='No Security Restore')
lines += ax2.plot(trustzone_time_x, trustzone_time_chkp, label='TrustZone Checkpoint')
lines += ax2.plot(trustzone_time_x, trustzone_time_rest, label='TrustZone Restore')

for i in range(len(lines)):
    lines[i].set_markevery(everies[i])
    lines[i].set_linestyle(linestyles[i])
    lines[i].set_marker(markers[i])
#     lines[i].set_color(colors[i])
ax2.legend(handles=lines)
# ax2.set_yscale('log')
ax2.grid()
fig_time.tight_layout()
fig_time.show()

In [49]:
tz_flash_write_charge = pickle.load(open(path.join("Validation", "No_Security_Flash", "no_security_flash_write_charge.p"), "rb"))
tz_flash_read_charge = pickle.load(open(path.join("Validation", "No_Security_Flash", "no_security_flash_read_charge.p"), "rb"))
tz_flash_write_time = pickle.load(open(path.join("Validation", "No_Security_Flash", "no_security_flash_write_time.p"), "rb"))
tz_flash_read_time = pickle.load(open(path.join("Validation", "No_Security_Flash", "no_security_flash_read_time.p"), "rb"))
tz_no_security = [tz_flash_write_charge, tz_flash_read_charge, tz_flash_write_time, tz_flash_read_time]
for i in tz_no_security:
    print(len(i))
    
tz_no_security_chkp = []
tz_no_security_rest = []
tz_no_security_x = []
for i in range(0, min_num):
    tz_no_security_x.append(i+1)
    tz_no_security_chkp.append(tz_flash_write_charge[i] * voltage * j_scale)
    tz_no_security_rest.append(tz_flash_read_charge[i] * voltage * j_scale)

6000
6000
6000
6000


In [54]:
fig_comp = plt.figure(figsize=(6, 4))
fig_comp.canvas.set_window_title("Analysis Compare")

In [55]:
fig_comp.clf()
ax1 = fig_comp.add_subplot(1, 1, 1)
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [$\mu$J]') # $\mu$

Text(0, 0.5, 'Energy [$\\mu$J]')

In [56]:
lines = []
lines += ax1.plot(tz_no_security_x, tz_no_security_chkp, label='TZ No Security Checkpoint')
lines += ax1.plot(tz_no_security_x, tz_no_security_rest, label='TZ No Security Restore')
lines += ax1.plot(no_security_x, no_security_chkp, label='No Security Checkpoint')
lines += ax1.plot(no_security_x, no_security_rest, label='No Security Restore')
for i in range(len(lines)):
    lines[i].set_markevery(everies[i+2])
    lines[i].set_linestyle(linestyles[i])
    lines[i].set_marker(markers[i])
#     lines[i].set_color(colors[i])

ax1.legend(handles=lines)
# ax1.set_yscale('log')
ax1.grid()
fig_comp.tight_layout()
fig_comp.show()